In [ ]:
from IPython.display import display, clear_output, Javascript, Markdown
import ipywidgets as ipw
import utils
import json
import google.generativeai as genai

In [5]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.get_aiidalab_eln_config()
# CONFIG_ELN = utils.read_json("eln_config.json")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])

prompt_textarea = utils.Textarea(
    description = "Prompt", 
    layout = ipw.Layout(width = '980px', height = '300px'),
    style = {"description_width": "110px"}
)

model_answer_textarea = utils.Textarea(
    description = "Answer", 
    layout = ipw.Layout(width = '980px', height = '300px'),
    style = {"description_width": "110px"}
)

enter_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Enter', icon = 'arrow-right', layout = ipw.Layout(width = '100px', height = '50px')
)

quit_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Main menu', icon = 'home', layout = ipw.Layout(width = '100px', height = '50px')
)

increase_buttons_size = utils.HTML(data = ''.join(CONFIG["save_home_buttons_settings"]))

# Google Gemini 1.5 Flash
google_api_key = utils.read_json("/home/jovyan/gemini_api.json")
genai.configure(api_key=google_api_key["api_key"])
model_name = "gemini-1.5-flash"
system_instruction = "You are a materials science expert working with nanotech materials."
model = genai.GenerativeModel(
    model_name = model_name, 
    system_instruction = system_instruction
)

# Start the chatbot
chat = model.start_chat()

In [ ]:
def close_notebook(b):
    display(utils.Javascript(data = 'window.location.replace("home.ipynb")'))

def ask_chatbot(change):
    model_answer_textarea.value = ""
    prompt = prompt_textarea.value
    response = chat.send_message(prompt, stream = True)
    for chunk in response:
        model_answer_textarea.value = model_answer_textarea.value + chunk.text

def load_chatbot():
    openbis_objects = OPENBIS_SESSION.get_objects(attrs = ['parents'])
    openbis_objects_history = {}
    selected_objects = []
    for object in openbis_objects:
        object_props = object.props.all()
        # for prop in object_props:
        #     if OPENBIS_SESSION.get_property_type(prop).dataType == "SAMPLE" and object_props[prop] is not None:
        #         prop_object = OPENBIS_SESSION.get_object(object_props[prop])
        #         object_props[prop] = {"perm_id": object_props[prop]}
        #         object_props[prop]["type"] = str(prop_object.type)
        #         object_props[prop]["registrationDate"] = prop_object.registrationDate
        #         object_props[prop].update(prop_object.props.all())
        #         object_props[prop], openbis_objects_history = utils.get_parent_child_relationships_nested(OPENBIS_SESSION, prop_object, object_props[prop], openbis_objects_history)

        object_props["perm_id"] = object.permId
        object_props["type"] = str(object.type)
        object_props["registrationDate"] = object.registrationDate
        
        selected_object_schema, object_history = utils.get_parent_child_relationships_nested(
            OPENBIS_SESSION, 
            object, 
            object_props, 
            openbis_objects_history
        )
        
        selected_objects.append(selected_object_schema)
        openbis_objects_history.update(object_history)
    
    utils.create_json(selected_objects, "/home/jovyan/aiida-openbis/Notebooks/connection_to_openbis/data.json")
    json_string = json.dumps(selected_objects)
    history = [
        {"role": "user", "parts": [{"text": f"Hello, today is 20-12-2024. Consider this string as a database of knowledge of openBIS, an ELN designed to save research data. It contains a list with several JSON objects: {json_string} \n. Be ready to answer questions about this."}]},
        {"role": "model", "parts": [{"text": "Hi, ok feel free to ask me questions about that."}]}
    ]
    chat.history = history

In [ ]:
# myfile = genai.upload_file("/home/jovyan/aiida-openbis/Notebooks/connection_to_openbis/data.txt")

# cache = genai.caching.CachedContent.create(
#     model=model_name,
#     system_instruction=system_instruction,
#     contents=[myfile],
# )

# model = genai.GenerativeModel.from_cached_content(cached_content=cache)
# chat = model.start_chat()
# result = chat.send_message(["How many publications are in this JSON document?", myfile])

# openBIS chatbot

In [ ]:
load_chatbot()

In [ ]:
display(increase_buttons_size)
display(prompt_textarea)
display(enter_button)
display(model_answer_textarea)
display(quit_button)
enter_button.on_click(ask_chatbot)
quit_button.on_click(close_notebook)